In [29]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

#ファイル読み込み
im1 = np.array(Image.open('image1.png'))
im2 = np.array(Image.open('image2.png'))

#配列を一次元化
x1 = im1.flatten()
x2 = im2.flatten()

#平均0にする
x1 = x1 - np.mean(x1)
x2 = x2 - np.mean(x2)
x = [x1,x2]

#共分散行列を求める
xcov = np.cov(x,rowvar = 1,bias = 1)
mx = np.asmatrix(xcov)

#固有値計算、白色化
t,E = np.linalg.eig(mx)
tv = 1 / (t ** (1/2) )
Dv = np.diag(tv)
Dv = np.asmatrix(Dv)
V = E * Dv * E.T
z = V * x

In [30]:
#正規化
def normalize(vec):
    vec_norm = np.linalg.norm(vec)
    if vec.sum()<0:
        vec *= -1
    return vec / vec_norm

#変換
def convert(w,z):
    step1 = np.asarray(z) * (np.asarray(w.T * z)**3)
    step2 = np.mean(step1,axis=1)
    step3 = np.asmatrix(step2).T - 3*w
    return np.asarray(step3)

In [31]:
#尖度を最大化するWを求める
W = np.empty((0,2))
for i in range(2):
    w = np.random.rand(2,1)
    w = normalize(w)
    while True:
        tmp = w
        w = convert(w,z)
        w = normalize(w)
        if np.linalg.norm(w-tmp)<0.000001:
            W = np.concatenate((W,w.T),axis=0)
            break

In [37]:
y = W * z
y = np.asarray(y)
image01 = y[0]
image02 = y[1]

In [60]:
#配列の長さ
l = len(im1.tolist())

#配列要素を0-255に調整
i1 = (image01 - np.min(image01))
image001 = i1*255/np.max(i1) 
i2 = (image02 - np.min(image02))
image002 = i2*255/np.max(i2) 

#配列の復元
image1 = image001.reshape(l,l)
image2 = image002.reshape(l,l)

#画像の書き出し
Image.fromarray(np.uint8(image1)).save('reimage1.png')
Image.fromarray(np.uint8(image2)).save('reimage2.png')